### 예측 모델 수행하기

고객 10866건에 대한 학습용 데이터를 이용하여 자전거 대여량 예측 모형을 만든다.<br>
생성한 예측 모형으로 평가용 데이터에 해당하는 6493건의 자전거 대여량 예측값을 다음과 같은 형식은 CSV 파일로 생성하시오.<br>
제출한 모델의 성능은 R^2 score 평가지표에 따라 채점

### 3.1 데이터 탐색하기 

데이터 가져오기

In [1]:
import pandas as pd

x_train = pd.read_csv('bike_x_train.csv', encoding='CP949')
x_test = pd.read_csv('bike_x_test.csv' , encoding = 'CP949')
y_train = pd.read_csv('bike_y_train.csv')

In [2]:
# 독립변수인 x_train, x_test의 상위 3개 행을 확인하기
print(x_train.head(3))

          datetime  계절  공휴일  근무일  날씨    온도    체감온도  습도   풍속
0  2011-01-01 0:00   1    0    0   1  9.84  14.395  81  0.0
1  2011-01-01 1:00   1    0    0   1  9.02  13.635  80  0.0
2  2011-01-01 2:00   1    0    0   1  9.02  13.635  80  0.0


In [3]:
print(x_test.head(3))
#날짜 유형인 datetime 칼럼과 연속형 변수인 온도,체감온도,습도,풍속 칼럼이 있고 범주형은 계절,공휴일,근무일,날씨가 있음

          datetime  계절  공휴일  근무일  날씨     온도    체감온도  습도       풍속
0  2011-01-20 0:00   1    0    1   1  10.66  11.365  56  26.0027
1  2011-01-20 1:00   1    0    1   1  10.66  13.635  56   0.0000
2  2011-01-20 2:00   1    0    1   1  10.66  13.635  56   0.0000


In [4]:
print(y_train.head(3)) # datetime 칼럼과 해당시간에 자전거 대여 수량인 count 칼럼이 있다. 

          datetime  count
0  2011-01-01 0:00     16
1  2011-01-01 1:00     40
2  2011-01-01 2:00     32


행/열 확인하기

In [5]:
# 각 데이터 세트의 행과 열 개수를 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(10886, 9)
(6493, 9)
(10886, 2)


요약정보 확인하기

In [6]:
# x_train 세트의 요약정보 확인하기
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  10886 non-null  object 
 1   계절        10886 non-null  int64  
 2   공휴일       10886 non-null  int64  
 3   근무일       10886 non-null  int64  
 4   날씨        10886 non-null  int64  
 5   온도        10886 non-null  float64
 6   체감온도      10886 non-null  float64
 7   습도        10886 non-null  int64  
 8   풍속        10886 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 765.5+ KB
None


In [7]:
# 범주형 변수들의 값을 확인하기
print(x_train['계절'].unique())
print(x_train['공휴일'].unique())
print(x_train['근무일'].unique())
print(x_train['날씨'].unique())

[1 2 3 4]
[0 1]
[0 1]
[1 2 3 4]


기초 통계량 확인하기

In [8]:
# x_train의 기초 통계량을 확인하고, 가독성을 위해 행/열 바꿔서 출력하기
print(x_train.describe().T)

        count       mean        std   min      25%     50%      75%       max
계절    10886.0   2.506614   1.116174  1.00   2.0000   3.000   4.0000    4.0000
공휴일   10886.0   0.028569   0.166599  0.00   0.0000   0.000   0.0000    1.0000
근무일   10886.0   0.680875   0.466159  0.00   0.0000   1.000   1.0000    1.0000
날씨    10886.0   1.418427   0.633839  1.00   1.0000   1.000   2.0000    4.0000
온도    10886.0  20.230860   7.791590  0.82  13.9400  20.500  26.2400   41.0000
체감온도  10886.0  23.655084   8.474601  0.76  16.6650  24.240  31.0600   45.4550
습도    10886.0  61.886460  19.245033  0.00  47.0000  62.000  77.0000  100.0000
풍속    10886.0  12.799395   8.164537  0.00   7.0015  12.998  16.9979   56.9969


독립변수와 종속변수의 관계 확인하기<br>
둘의 관계를 파악하여 자전거 대여량에 영향을 미치는 변수 확인하기

In [9]:
# x_train, y_train을 세로 방향으로 통합한 후 data 변수에 저장하기
data =pd.concat([x_train,y_train],axis=1)

In [10]:
#계절 칼럼에 따른 count(자전거 대여량)합계를 구하기
# 1(봄),2(여름),3(가을),4(겨울)
print(data.groupby(['계절'])['count'].sum())
# 가을이 봄의 2배

계절
1    312498
2    588282
3    640662
4    544034
Name: count, dtype: int64


In [11]:
#공휴일 칼럼에 따른 count 합계 구하기
# 0(공휴일아님),1(공휴일)
print(data.groupby(['공휴일'])['count'].sum())
# 공휴일 아닌 날이 앞도적으로 높다

공휴일
0    2027668
1      57808
Name: count, dtype: int64


In [12]:
#근무일 칼럼에 따른 count 합계 구하기
#0(근무일 아님),1(근무일)
print(data.groupby(['근무일'])['count'].sum())
#근무일 대여량이 더 높다

근무일
0     654872
1    1430604
Name: count, dtype: int64


In [13]:
#날씨 칼럼에 따른 count 합계 구하기
#1(아주 깨끗한 날씨),2(안개와 구름 있는),3(조금의 눈과 비 또는 조금의 천둥),4(아주 많은 비 또는 우박)
print(data.groupby(['날씨'])['count'].sum())
#날씨가 좋을 수록 대여량이 높다

날씨
1    1476063
2     507160
3     102089
4        164
Name: count, dtype: int64


In [14]:
# 각 독립변수들이 자전거 대여량에 영향을 미침

### 3.2 전처리하기 

파생변수 만들기<br>
datetime 칼럼은 연도 월 요일 시간대로 분리 가능 <br>
년, 월, 일, 시간으로 데이터 분리하고 요일 데이터 새로 만들기

In [15]:
# datetime 칼럼의 데이터 타입을 날짜 타입으로 변환하기
x_train['datetime']=pd.to_datetime(x_train['datetime'])

In [16]:
#x_train의 datetime 칼럼에서 연도 데이터를 추출하여 year 칼럼에 저장하기
x_train['datetime']=pd.to_datetime(x_train['datetime'])

In [17]:
#x_train의 year에서 중복 제거 값 확인하기
print(x_train['year'].unique())

[2011 2012]


In [18]:
#x_train의 datetime 칼럼에서 월 데이터를 추출하여 month 칼럼에 저장하기
x_train['month']=x_train['datetime'].dt.month
#x_train의 month에서 중복 제거 값 확인하기
print(x_train['month'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12]


In [19]:
#x_train의 datetime 칼럼에서 일 데이터를 추출하여 day 칼럼에 저장하기
x_train['day']=x_train['datetime'].dt.day
#x_train의 day에서 중복 제거 값 확인하기
print(x_train['day'].unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [20]:
#x_train의 datetime 칼럼에서 시간 데이터를 추출하여 hour 칼럼에 저장하기
x_train['hour']=x_train['datetime'].dt.hour
#x_train의 hour에서 중복 제거 값 확인하기
print(x_train['hour'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [21]:
#x_train의 datetime 칼럼에서 요일 데이터를 추출하여 dayofweek 칼럼에 저장하기
x_train['dayofweek']=x_train['datetime'].dt.dayofweek
#x_train의 dayofweek에서 중복 제거 값 확인하기
#0(월),1(화),2(수),3(목),4(금),5(토),6(일)
print(x_train['dayofweek'].unique())

[5 6 0 1 2 3 4]


x_trian의 연도 월 일 시간 요일에 대한 파생변수를 생성한 후 y_train 세트를 세로방향으로 통합하여 추가 분석 진행

In [22]:
# 파생변수가 포함된 독립변수와 종속변수를 통합한 data2 만들기
data2=pd.concat([x_train,y_train],axis=1)

새로 만든 'year', 'month','day','hour','dayofweek' 칼럼별로 자전거 대여량의 합계를 계산한 후 <br>
각 독립변수에 따른 count 값의 추세를 확인

In [23]:
# year 칼럼에 따른 count 합계 구하기
print(data2.groupby(['year'])['count'].sum())
#2012년이 2011년보다 두배 정도 많음

year
2011     781979
2012    1303497
Name: count, dtype: int64


In [24]:
# month 칼럼에 따른 count 합계 구하기
print(data2.groupby(['month'])['count'].sum())
#1,2월은 적은 편이고 3-12월은 상대적으로 대여량이 증가함 그러나 특정한 대여량이 월등하지 않고 비슷하므로 분석 대상에서 제외
x_train=x_train.drop(columns=['month'])

month
1      79884
2      99113
3     133501
4     167402
5     200147
6     220733
7     214617
8     213516
9     212529
10    207434
11    176440
12    160160
Name: count, dtype: int64


In [25]:
# day 칼럼에 따른 count 합계 구하기
print(data2.groupby(['day'])['count'].sum())
#1일부터 19일 까지 모두 비슷한 값이므로 분석대상에서 제외
x_train=x_train.drop(columns=['day'])

day
1     103692
2     105381
3     111561
4     112335
5     109115
6     108600
7     105486
8     102770
9     108041
10    111645
11    111146
12    109257
13    111448
14    112406
15    115677
16    109837
17    118255
18    108437
19    110387
Name: count, dtype: int64


In [26]:
# hour 칼럼에 따른 count 합계 구하기
print(data2.groupby(['hour'])['count'].sum())
#출근 시간대인 8-9시 퇴근시간인 17-18시에 자전거 대여량이 상대적으로 높아짐

hour
0      25088
1      15372
2      10259
3       5091
4       2832
5       8935
6      34698
7      96968
8     165060
9     100910
10     79667
11     95857
12    116968
13    117551
14    111010
15    115960
16    144266
17    213757
18    196472
19    143767
20    104204
21     79057
22     60911
23     40816
Name: count, dtype: int64


In [27]:
# dayofweek 칼럼에 따른 count 합계 구하기
print(data2.groupby(['dayofweek'])['count'].sum())
# 각 요일별 큰 차이가 없어 분석대상에서 제외
x_train=x_train.drop(columns=['dayofweek'])

dayofweek
0    295296
1    291985
2    292226
3    306401
4    302504
5    311518
6    285546
Name: count, dtype: int64


파생변수 생성 과정은 테스트 데이터 세트에도 동일 적용

In [28]:
x_test['datetime']=pd.to_datetime(x_test['datetime'])
x_test['year']=x_test['datetime'].dt.year
x_test['hour']=x_test['datetime'].dt.hour

불 필요한 칼럼 삭제하기

In [29]:
#x_test의 datetime 칼럼 값은 x_test_datetime 변수에 저장하기
x_test_datetime = x_test['datetime']

In [30]:
#x_train,x_test에서 datetime 칼럼을 삭제하기
x_train = x_train.drop(columns=['datetime'])
x_test = x_test.drop(columns=['datetime'])
y_train = y_train.drop(columns=['datetime'])

### 3.3 학습하고 평가하기

데이터 분리하기

In [31]:
# 데이터 분리하기 위한 train_test_split 함수 가져오기
from sklearn.model_selection import train_test_split

In [32]:
#학습용과 검증용을 8:2로 분리하여 각 4개의 변수에 저장하기
X_TRAIN,X_TEST,Y_TRAIN,Y_TEST = train_test_split(x_train,y_train,\
                                                test_size=0.2, random_state=10)

In [33]:
#분리된 데이터의 행/열 구조 확인
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(8708, 10)
(2178, 10)
(8708, 1)
(2178, 1)


데이터 학습 및 하이퍼 파라미터 튜닝하기<br>
예측모델에서 주로 사용하는 XGB회귀를 활용하여 데이터를 학습할 모델 생성

In [34]:
#xgboost 라이브러리에서 XGBRegressor 함수 가져오기
from xgboost import XGBRegressor

In [35]:
# XGB 회귀 분석으로 수행할 첫 번째 모델을 만들고, 공부시키기
model = XGBRegressor(n_estimator=100, max_depth=3, random_state=10)
model.fit(X_TRAIN,Y_TRAIN)

[16:10:23] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimator=100, n_estimators=100,
             n_jobs=4, num_parallel_tree=1, predictor='auto', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [36]:
#XGB 회귀 분석으로 수행할 두번 째 모델을 만들고 공부시키기
model2= XGBRegressor(n_estimators=200,max_depth=5, random_state=10)
model2.fit(X_TRAIN,Y_TRAIN)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

결과 예측하기<br>
종속변수인 count값을 예측하기 위해 다음 코드의 predict() 함수 사용<br>
즉 문제에서 주어진 테스트 데이터(x_test)로 predict를 수행하여 y_test를 예측(y_test_predicted)하는 것<br>
반드시 pd.DateFrame() 함수를 사용하여 데이터 타입 변경<br>
최종 제출할 칼럼명은 'count'이므로 칼럼명 0을 count 문자열로 설정 칼럼명 변경코드는 rename(columns={0:'count'})입니다

In [37]:
# 학습이 완료된 모델을 통해 y_test 값을 예측하기 :최종 결과 제출용
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'count'})
y_test_predicted2 = pd.DataFrame(model2.predict(x_test)).rename(columns={0:'count'})

In [38]:
#y_test_predicted의 count 칼럼 값이 음수인 데이터를 추출하여 0으로 바꾸기
y_test_predicted[y_test_predicted['count']<0]=0
y_test_predicted2[y_test_predicted2['count']<0]=0

In [39]:
#y_test_predicted에서 상위 10개 행을 확인하기
print(y_test_predicted.head(10))
print(y_test_predicted2.head(10))

        count
0    2.139193
1    0.979986
2    0.000000
3    0.000000
4    0.000000
5    7.486256
6   79.849319
7  141.700363
8  319.266785
9   91.251663
        count
0   21.486200
1    0.061608
2    0.000000
3    2.039008
4    2.553952
5   10.101068
6   33.650478
7  109.718521
8  248.734039
9  145.416061


향후 R^2 평가지표를 계산하기 위해 X_TEST 검증용 세트에 대응하는 Y_TEST 예측값을 계산<br>
예측된 Y_TEST 값은 Y_TEST_PREDICTED 변수에 저장
Y_TEST_PREDICTED 는 자전거 대여량을 의미하기 때문에 만약 음수가 존재한다면 0으로 바꾸기

In [40]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기 : 평가지표 계산용
Y_TEST_PREDICTED=pd.DataFrame(model.predict(X_TEST)).rename(columns={0:'count'})
Y_TEST_PREDICTED2=pd.DataFrame(model2.predict(X_TEST)).rename(columns={0:'count'})

In [41]:
# Y_TEST_PREDICTED의 count 값이 음수인 데이터를 0으로 바꾸기
Y_TEST_PREDICTED[Y_TEST_PREDICTED['count']<0]=0
Y_TEST_PREDICTED2[Y_TEST_PREDICTED2['count']<0]=0

모델 평가하기

In [42]:
# sklearn 패키지의 metrics 모듈에서 r2_score 함수를 가져오기
from sklearn.metrics import r2_score

In [43]:
#1차 학습 모델의 R^2 평가지표 값을 확인하기
print(r2_score(Y_TEST,Y_TEST_PREDICTED))

0.9044130641900863


In [44]:
#2차 학습 모델의 R^2 평가지표 값을 확인하기
print(r2_score(Y_TEST,Y_TEST_PREDICTED2))

0.9381182890431057


두 번째 모델의 평가 지표결과가 더 높으므로 2번째 모델 선택

### 3.4 결과 제출하기
최종 데이터는 datetime 칼럼과 y_test 세트의 예측값이 포함 되어야 한다 <br>
따라서 x_test_datetime 변수와 y_test_predicted2 변수를 세로 방향으로 통합

In [46]:
#x_test_datetime 변수와 y_test_predicted2 변수를 세로 방향으로 붙이기
print(pd.concat([x_test_datetime,y_test_predicted2],axis=1))

                datetime       count
0    2011-01-20 00:00:00   21.486200
1    2011-01-20 01:00:00    0.061608
2    2011-01-20 02:00:00    0.000000
3    2011-01-20 03:00:00    2.039008
4    2011-01-20 04:00:00    2.553952
...                  ...         ...
6488 2012-12-31 19:00:00  204.254379
6489 2012-12-31 20:00:00  142.610077
6490 2012-12-31 21:00:00  132.809982
6491 2012-12-31 22:00:00   92.404938
6492 2012-12-31 23:00:00   38.964684

[6493 rows x 2 columns]


In [47]:
# 앞의 출력 결과를 final 변수에 저장하기
final = pd.concat([x_test_datetime,y_test_predicted2],axis=1)

In [48]:
# final 변수를 1234567.csv 이름으로 저장하기
final.to_csv('1234567.csv',index=False)